## Ejercicio 9

Utilizando los ejemplos del archivo **AUTOS.csv** genere un modelo utilizando un multiperceptrón para predecir el precio del auto (atributo **price**) y la cantidad de millas por galón en ruta (**MPG-highway**) en función del resto de los atributos. Recuerde completar los valores faltantes, utilizar normalización y dividir el dataset en entrenamiento y validación (80/20).

Realice 20 ejecuciones independientes de cada configuración seleccionada calculando las épocas promedio y al error cuadrático medio (ECM). Analice los resultados y respalde las afirmaciones referidas a los resultados obtenidos. Utilice un máximo de 1000 épocas con lotes de 50 e implemente una parada temprana con paciencia de 15.

Complete la siguiente tabla y realice un análisis de los valores obtenidos:

<table style="border-collapse: collapse; width: 100%; text-align: center;">
  <thead>
    <tr style="background-color:#2f8fbd; color: white;">
      <th>Optimizador</th>
      <th>Función activación</th>
      <th>Épocas promedio</th>
      <th>ECM Promedio</th>
    </tr>
  </thead>
  <tbody>
    <tr style="background-color:#2f8fbd; color: white;">
      <td rowspan="4">SGD</td>
      <td>tanh</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>sigmoid</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>ReLU</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>LeakyReLU</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td rowspan="4">RMSProp</td>
      <td>tanh</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>sigmoid</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>ReLU</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>LeakyReLU</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td rowspan="4">Adam</td>
      <td>tanh</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>sigmoid</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>ReLU</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
    <tr style="background-color:#2f8fbd; color: white;">
      <td>LeakyReLU</td>
      <td style="background-color:white; color:black;"></td>
      <td style="background-color:white; color:black;"></td>
    </tr>
  </tbody>
</table>



Donde:

- **Épocas Promedio** es el número de épocas promedio en el que se detuvo el entrenamiento.
- **ECM Promedio** es el promedio del error de predicción en cada entrenamiento.


##### Cargar datos

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

DATOS_DIR = "../../Datos/"
data = pd.read_csv(DATOS_DIR + 'autos.csv')

##### Eliminar valores faltantes

In [3]:
data = data.replace('?', np.nan)
# print(data.isnull().sum())
cols = ["normalized-losses", "bore", "stroke", "horsepower", "peak-rpm", "price"]

for col in cols:
    data[col] = pd.to_numeric(data[col], errors="coerce")
    data[col] = data[col].fillna(data[col].mean())

data = data.select_dtypes(include = ["int16", "int32", "int64", "float16", "float32", "float64"])

##### Normalizar datos

In [4]:
from sklearn.preprocessing import StandardScaler

target_columns = [12, 14]  # mpg-highway y price
Y_raw = data.iloc[:, target_columns].values
X_raw = data.drop(data.columns[target_columns], axis=1).values

data_scaler, target_scaler = StandardScaler(), StandardScaler()

X_raw = data_scaler.fit_transform(X_raw)
Y_raw = target_scaler.fit_transform(Y_raw)

##### Entrenamiento y validación

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

RUNS_PER_CONFIG = 20
MAX_EPOCHS = 1000
BATCH_SIZE = 50
TEST_SIZE = 0.2
PATIENCE = 15

ENTRADAS = X_raw.shape[1]
SALIDAS = T_raw.shape[1]

# Separar datos
X_train, X_test, Y_train, Y_test = train_test_split(
    X_raw, Y_raw, test_size=TEST_SIZE, random_state=42
)

optimizers = ['sgd', 'rmsprop', 'adam']
activations = ['tanh', 'sigmoid', 'relu']  # sacamos 'leaky_relu' por ahora

def get_optimizer(name):
    if name == 'sgd':
        return SGD()
    elif name == 'rmsprop':
        return RMSprop()
    elif name == 'adam':
        return Adam()

for opt in optimizers:
    for act in activations:
        epochs_list = []
        mse_list = []

        for run in range(RUNS_PER_CONFIG):
            # Definir modelo
            model = Sequential()
            model.add(Input(shape=(ENTRADAS,)))
            model.add(Dense(6, activation=act))
            model.add(Dense(3, activation=act))
            model.add(Dense(SALIDAS))

            # OJO: crear nueva instancia de optimizador cada vez
            model.compile(optimizer=get_optimizer(opt),
                          loss='mae', metrics=['mae', 'mse'])

            early_stop = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss', patience=PATIENCE, restore_best_weights=True
            )

            history = model.fit(
                x=X_train, y=Y_train,
                batch_size=BATCH_SIZE,
                epochs=MAX_EPOCHS,
                validation_data=(X_test, Y_test),
                callbacks=[early_stop],
                verbose=0
            )

            # Guardar número de épocas y MSE validación
            epochs_list.append(len(history.epoch))
            pred = model.evaluate(X_test, Y_test, verbose=0)
            mse_list.append(pred[2])  # MSE

        # Promedios
        mean_epochs = np.mean(epochs_list)
        mean_mse = np.mean(mse_list)

        print(f"\nConfig -> Optimizador: {opt}, Activación: {act}")
        print(f" - Promedio de épocas: {mean_epochs:.2f}")
        print(f" - Promedio de MSE (validación): {mean_mse:.5f}")



Config -> Optimizador: sgd, Activación: tanh
 - Promedio de épocas: 460.75
 - Promedio de MSE (validación): 0.38076

Config -> Optimizador: sgd, Activación: sigmoid
 - Promedio de épocas: 819.35
 - Promedio de MSE (validación): 0.70678

Config -> Optimizador: sgd, Activación: relu
 - Promedio de épocas: 540.35
 - Promedio de MSE (validación): 0.42910

Config -> Optimizador: rmsprop, Activación: tanh
 - Promedio de épocas: 360.15
 - Promedio de MSE (validación): 0.29857

Config -> Optimizador: rmsprop, Activación: sigmoid
 - Promedio de épocas: 741.45
 - Promedio de MSE (validación): 0.37088

Config -> Optimizador: rmsprop, Activación: relu
 - Promedio de épocas: 365.65
 - Promedio de MSE (validación): 0.33686

Config -> Optimizador: adam, Activación: tanh
 - Promedio de épocas: 326.70
 - Promedio de MSE (validación): 0.31950

Config -> Optimizador: adam, Activación: sigmoid
 - Promedio de épocas: 781.35
 - Promedio de MSE (validación): 0.34319

Config -> Optimizador: adam, Activación:

## Para Leaky-relu

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, LeakyReLU
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

RUNS_PER_CONFIG = 20
MAX_EPOCHS = 1000
BATCH_SIZE = 50
TEST_SIZE = 0.2
PATIENCE = 15

ENTRADAS = X_raw.shape[1]
SALIDAS = Y_raw.shape[1]

# Separar datos
X_train, X_test, Y_train, Y_test = train_test_split(
    X_raw, Y_raw, test_size=TEST_SIZE, random_state=42
)

optimizers = ['sgd', 'rmsprop', 'adam']
activation = 'leaky_relu'

def get_optimizer(name):
    if name == 'sgd':
        return SGD()
    elif name == 'rmsprop':
        return RMSprop()
    elif name == 'adam':
        return Adam()

for opt in optimizers:
    epochs_list = []
    mse_list = []

    for run in range(RUNS_PER_CONFIG):
        # Definir modelo con LeakyReLU
        model = Sequential()
        model.add(Input(shape=(ENTRADAS,)))
        model.add(Dense(6))
        model.add(LeakyReLU())
        model.add(Dense(3))
        model.add(LeakyReLU())
        model.add(Dense(SALIDAS))

        model.compile(optimizer=get_optimizer(opt),
                      loss='mae', metrics=['mae', 'mse'])

        early_stop = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=PATIENCE, restore_best_weights=True
        )

        history = model.fit(
            x=X_train, y=Y_train,
            batch_size=BATCH_SIZE,
            epochs=MAX_EPOCHS,
            validation_data=(X_test, Y_test),
            callbacks=[early_stop],
            verbose=0
        )

        # Guardar número de épocas y MSE validación
        epochs_list.append(len(history.epoch))
        pred = model.evaluate(X_test, Y_test, verbose=0)
        mse_list.append(pred[2])  # MSE

    # Promedios
    mean_epochs = np.mean(epochs_list)
    mean_mse = np.mean(mse_list)

    print(f"\nConfig -> Optimizador: {opt}, Activación: LeakyReLU")
    print(f" - Promedio de épocas: {mean_epochs:.2f}")
    print(f" - Promedio de MSE (validación): {mean_mse:.5f}")

2025-10-11 11:32:51.686884: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected



Config -> Optimizador: sgd, Activación: LeakyReLU
 - Promedio de épocas: 530.35
 - Promedio de MSE (validación): 0.28255

Config -> Optimizador: rmsprop, Activación: LeakyReLU
 - Promedio de épocas: 275.65
 - Promedio de MSE (validación): 0.23974

Config -> Optimizador: adam, Activación: LeakyReLU
 - Promedio de épocas: 286.40
 - Promedio de MSE (validación): 0.25809
